<a href="https://colab.research.google.com/github/shreyasat27/quantupid_10/blob/main/code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First of all we will be needing to install qiskit

In [ ]:
!pip install qiskit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.4/39.4 MB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 12.3 MB/s eta 0:00:00


This cell is performing the following work:

1. **Reading the Hamiltonian Data**: The code reads Hamiltonian terms from a text file, parsing coefficients and Pauli terms.

2. **Sorting Hamiltonian Terms**: The terms are sorted based on the absolute value of their coefficients, with higher absolute values considered more important.

3. **Ideal Unitary Computation**: It computes the ideal unitary operator `U_perfect` corresponding to the Hamiltonian using `scipy.linalg.expm`.

4. **Division of Hamiltonian**: The Hamiltonian is divided into two parts: `Z_part` containing terms with only I and Z operators, and `R_part` containing the remaining terms with X or Y operators.

5. **Circuit Synthesis Functions**: It defines a function `firstorder` to generate a circuit that trotterizes the evolution of a given sparse Pauli operator, and a function `greedy_sorter` to greedily sort the terms for efficient execution.

6. **Construction of Circuits**: Two smaller circuits are defined: `Z_part_evol_half` for implementing the unitary corresponding to `Z_part`, and `R_part_evol_one` for trotterizing the terms in `R_part`. The final circuit is composed of these smaller circuits.

7. **Transpilation**: The final circuit is transpiled for optimization using Qiskit's `transpile` function.

8. **Precision Checking**: The precision of the produced circuit is checked by comparing it with the ideal unitary operator `U_perfect`. Special care is taken to handle global phase issues introduced during transpilation.

9. **Results**: The code prints out the operator norm error and the depth of the circuit.

In [ ]:
import numpy as np
import qiskit
import scipy
from scipy import linalg
from qiskit.quantum_info import SparsePauliOp, Operator, Pauli
from qiskit.circuit.library import PauliEvolutionGate
from qiskit.synthesis import LieTrotter, SuzukiTrotter
from qiskit import transpile

######## Reading the data
# './hamiltonian_string_raw.txt' should contain the Hamiltonian
# posted to the competition forum
hamiltonian_terms = []
with open('/content/hamiltonian_string_raw.txt', 'r') as f:
    for line in f:
        line_remove_newlines = line.replace('\n', '')
        line_remove_spaces = ''.join(line_remove_newlines.split(' '))
        if line_remove_spaces == '':
            continue
        line_items = line_remove_spaces.split('*')
        assert len(line_items) == 2
        coef, pauli_term = line_items
        hamiltonian_terms.append((float(coef), pauli_term))

######## Sort by absolute value of the coefficient.
# Terms with high absolute value of the coefficient are more important

hamiltonian_terms.sort(key=lambda term: abs(term[0]))
for term in hamiltonian_terms:
    print(term[0], '\t', term[1])
print(f'Hamiltonian has {len(hamiltonian_terms)} terms')

######## Compute U_perfect := the ideal unitary that we should simulate
H = SparsePauliOp.from_list([(item[1], item[0]) for item in hamiltonian_terms])
U_perfect = scipy.linalg.expm(-1j * H.to_matrix())

######## Divide the hamiltonian into two parts:
# Z_part, which contains terms with Paulis made up only from I and Z operators
# R_part, which contains all the rest
Z_part = SparsePauliOp.from_list([(item[1], item[0]) for item in hamiltonian_terms if 'X' not in item[1] and 'Y' not in item[1]])
R_part = SparsePauliOp.from_list([(item[1], item[0]) for item in hamiltonian_terms if 'X' in item[1] or 'Y' in item[1]])

######## Helper function which takes a SparsePauliOp, time t,
# and returns a circuit that trotterizes the e^-it*sparsepauliop
def firstorder(sparsepauliop, t):
    UT = PauliEvolutionGate(sparsepauliop, t)
    trotter = LieTrotter(reps=1, cx_structure='fountain')
    return trotter.synthesize(UT)

######## A greedy sorter for a SparsePauliOp.
# Say, we have some Pauli terms IXXII, IIIZX. Then they can be trotterized efficiently,
# i.e. each works on different set of qubits so they can be executed in parallel.
# We try to order the terms so that such occurences are maximized.
def greedy_sorter(sparsepauliop):
    s = list(item for item in sparsepauliop)
    new_order = []
    while len(s) > 0:
        coll = []
        coll_used = set()
        added = True
        while added:
            added = False
            for item in s:
                item_pauli = item.paulis[0]
                item_used = set(i for i in range(len(item_pauli)) if item_pauli[i] != Pauli('I'))
                if len(coll_used.intersection(item_used)) == 0:
                    coll.append(item)
                    coll_used = coll_used.union(item_used)
                    s.remove(item)
                    added = True
        new_order = new_order + coll
    ret = new_order[0]
    for j in range(1, len(new_order)):
        ret = ret + new_order[j]
    return ret

######## Let us define two smaller circuits:
# Z_part_evol_half = a circuit that implements the unitary e^-0.5*Z_part
# R_part_evol_one = a circuit that trotterizes all the R_parts
# The final circuit will consist of three parts:
# (Z_part_evol_half) (R_part_evol_one) (Z_part_evol_half)
# It turns out that such circuit achieves an accuracy well below the required 0.1 threshold.
# So we simply ignore the terms from R_part with lowest abs. value of the coefficient,
# losing some of the precision but making the circuit shorter.
Z_part_evol_half = firstorder(greedy_sorter(Z_part), 0.5)
R_part_evol_one = firstorder(greedy_sorter(R_part[120:]), 1)
circ = Z_part_evol_half.compose(R_part_evol_one).compose(Z_part_evol_half)
circ_transpiled = transpile(circ, optimization_level=2, basis_gates=['x', 'y', 'z', 'rz', 'ry', 'rx', 'h', 'u', 'cx'])
# circ_transpiled.qasm3(filename='./solution.qasm')

######## Finally, we will check the precision of the produced circuit.
circ_loaded = circ_transpiled
U = np.asmatrix(qiskit.quantum_info.Operator(circ_loaded))

# Note that special care must be taken, since transpiling to QASM in qiskit
# messes up the global phase.
assert np.allclose(abs(U[0, 0]), 1)
U_fixed_phase = U / U[0, 0]
print(f"Operator norm error: {np.linalg.norm(U_perfect - U_fixed_phase, ord=2)}")
print(f"Circuit depth: {circ_loaded.depth()}")

# Results of the run:
# Operator norm error: 0.0986
# Circuit depth: 697


-0.25 	 XXI
0.25 	 XXZ
0.25 	 YYI
-0.25 	 YYZ
0.5 	 XXX
-0.5 	 XYY
-0.5 	 YXY
-0.5 	 YYX
Hamiltonian has 8 terms


QiskitError: 'Could not determine the number of qubits from an empty list. Try passing num_qubits.'

Here we will be drawing the circuit

In [ ]:
from qiskit.visualization import circuit_drawer

# Draw the circuit
circuit_drawer(circ, output='text')


┌─────────────────────┐┌─────────────────────┐┌─────────────────────┐»
q_0: ┤0                    ├┤0                    ├┤0                    ├»
     │                     ││                     ││                     │»
q_1: ┤1                    ├┤1                    ├┤1                    ├»
     │                     ││                     ││                     │»
q_2: ┤2                    ├┤2                    ├┤2                    ├»
     │                     ││                     ││                     │»
q_3: ┤3                    ├┤3                    ├┤3                    ├»
     │                     ││                     ││                     │»
q_4: ┤4                    ├┤4                    ├┤4                    ├»
     │  exp(it IIIZZIIIII) ││  exp(it IIIIIZIIZI) ││  exp(it ZZIIIIIIII) │»
q_5: ┤5                    ├┤5                    ├┤5                    ├»
     │                     ││                     ││                     │»
q_6: ┤6                    ├┤6                    ├┤6                    ├»
     │                     ││                     ││                     │»
q_7: ┤7                    ├┤7                    ├┤7                    ├»
     │                     ││                     ││                     │»
q_8: ┤8                    ├┤8                    ├┤8                    ├»
     │                     ││                     ││                     │»
q_9: ┤9                    ├┤9                    ├┤9                    ├»
     └─────────────────────┘└─────────────────────┘└─────────────────────┘»
«     ┌─────────────────────┐┌─────────────────────┐┌─────────────────────┐»
«q_0: ┤0                    ├┤0                    ├┤0                    ├»
«     │                     ││                     ││                     │»
«q_1: ┤1                    ├┤1                    ├┤1                    ├»
«     │                     ││                     ││                     │»
«q_2: ┤2                    ├┤2                    ├┤2                    ├»
«     │                     ││                     ││                     │»
«q_3: ┤3                    ├┤3                    ├┤3                    ├»
«     │                     ││                     ││                     │»
«q_4: ┤4                    ├┤4                    ├┤4                    ├»
«     │  exp(it IIIIIIZIIZ) ││  exp(it IIZIIIIZII) ││  exp(it IIIIIIIIII) │»
«q_5: ┤5                    ├┤5                    ├┤5                    ├»
«     │                     ││                     ││                     │»
«q_6: ┤6                    ├┤6                    ├┤6                    ├»
«     │                     ││                     ││                     │»
«q_7: ┤7                    ├┤7                    ├┤7                    ├»
«     │                     ││                     ││                     │»
«q_8: ┤8                    ├┤8                    ├┤8                    ├»
«     │                     ││                     ││                     │»
«q_9: ┤9                    ├┤9                    ├┤9                    ├»
«     └─────────────────────┘└─────────────────────┘└─────────────────────┘»
«     ┌─────────────────────┐┌─────────────────────┐┌─────────────────────┐»
«q_0: ┤0                    ├┤0                    ├┤0                    ├»
«     │                     ││                     ││                     │»
«q_1: ┤1                    ├┤1                    ├┤1                    ├»
«     │                     ││                     ││                     │»
«q_2: ┤2                    ├┤2                    ├┤2                    ├»
«     │                     ││                     ││                     │»
«q_3: ┤3                    ├┤3                    ├┤3                    ├»
«     │                     ││                     ││                     │»
«q_4: ┤4                    ├┤4                    ├┤4                    ├»
«     │  exp(it IIIIII